In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os, sys, glob
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras.backend as K
from IPython.display import Image
import random
import tqdm
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
imgdir = "../input/background-removed-happywhale-dataset/"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install visualkeras --upgrade

In [ ]:
sns.set_style("white")
import visualkeras as vk

In [ ]:
print(imgdir)
for file in glob.glob(os.path.join(imgdir, "*")): 
    print(f" \_ {file}")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras import layers

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train = pd.read_csv(imgdir + "seg_train.csv")
train = train[0:10000]
train.head()

In [ ]:
encoder = LabelEncoder()
train['species'].replace('kiler_whale', 'killer_whale', inplace=True)
train['species'].replace('bottlenose_dolpin', 'bottlenose_dolphin', inplace=True)
train['species'].replace(('globis', 'pilot_whale'), 'short_finned_pilot_whale', 
                         inplace=True)
train['species_id'] = encoder.fit_transform(train['species'])

In [ ]:
fig, ax = plt.subplots(dpi=200)
sns.histplot(data=train, x='species', hue='species', stat='percent', 
             discrete=True, palette="rocket")
plt.xticks(rotation=90)
plt.title("Distribution of different whale species")
plt.legend()
sns.despine()

In [ ]:
Image(filename= imgdir + "seg_img/" + random.choice(train['image']))

In [ ]:
with tqdm.tqdm(desc="processing", total=train.shape[0]) as progress:
    X = np.zeros((train.shape[0], 32, 32, 3))
    #X = np.zeros((train.shape[0], 64, 64, 3))
    for n, i in enumerate(train['image']):
        img = image.load_img(imgdir + 'seg_img/' + i, target_size=(32, 32, 3))
        #img = image.load_img(imgdir + 'seg_img/' + i, target_size=(64, 64, 3))
        X[n] = preprocess_input(image.img_to_array(img))
        progress.update(1)

In [ ]:
X = X.astype('float32') / 255.0
y = train['species_id']

In [ ]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    y = onehot_encoded
    return y, label_encoder

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=69)

In [ ]:
plt.imshow(X[0])

In [ ]:
y, label_encoder = prepare_labels(train['individual_id'])

In [ ]:
from tensorflow.keras.applications import NASNetLarge
base_model = NASNetLarge(
               input_shape=(32,32,3), 
               weights=None,
               include_top=False)

layer = base_model.output
layer = Flatten()(layer)
predictions = Dense(y.shape[1], activation='softmax')(layer)
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

In [ ]:
vk.layered_view(model)

In [ ]:
#best = "best.hdf5"
#callbacks = [
        #ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=1e-7),
        #EarlyStopping(monitor='val_accuracy', patience=5, min_delta=1e-5), 
        #ModelCheckpoint(best, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
    #]

In [ ]:
batch_size = [20, 50, 100, 256, 512]
epochs = [10, 20, 50, 100, 150]
#optimizer = ['RMSprop', 'Adam','SGD','Adadelta','Adagrad','Adamax','Nadam','Ftrl']
optimizer = ['RMSprop', 'Adam']
#neurons = [50,100]

def create_model(neurons=neurons, epochs=epochs, optimizer=optimizer, batch_size=batch_size):
    model = Sequential()
    model.add(Conv2D(128, 3, padding='same', activation='relu', input_shape=[32,32,3]))
    model.add(MaxPooling2D(3))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(256, activation='softmax'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='softmax'))
    model.add(Dropout(0.2))
    model.add(Dense(26, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

#model_BayesianOptimization = KerasClassifier(build_fn=create_model, verbose=0)
model_GridSearch = KerasClassifier(build_fn=create_model, verbose=0)
#model_RandomSearch = KerasClassifier(build_fn=create_model, verbose=0)

#param_opt = dict(batch_size=batch_size, epochs=epochs, neurons=neurons, optimizer = optimizer)

In [ ]:
#grid = GridSearchCV(estimator=model_GridSearch, param_grid=param_opt, n_jobs=1, cv=3, verbose = 0)
#grid_result = grid.fit(X_train, y_train)

In [ ]:
#print('according to gridsearch the best parameters are : ')
#print('batch_size : ' + str(grid_result.best_params_['batch_size']))
#print('epochs : ' + str(grid_result.best_params_['epochs']))
#print('neurons : ' + str(grid_result.best_params_['neurons']))
#print('optimizer : ' + str(grid_result.best_params_['optimizer']))

In [ ]:
#opt = Adam(learning_rate=1e-3)
#model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

#history = model.fit(X_train, y_train, batch_size = 20, epochs = 10,validation_data = (X_val, y_val))
history = model.fit(X, y, epochs=50, batch_size=128, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()